## Execution of 8 SQL queries from Python

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from pathlib import Path

In [2]:
# Configuration
load_dotenv()

MYSQL_USER     = os.getenv("MYSQL_USER",     "root")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD", "")
MYSQL_HOST     = os.getenv("MYSQL_HOST",     "localhost")
MYSQL_PORT     = os.getenv("MYSQL_PORT",     "3306")
MYSQL_DB       = os.getenv("MYSQL_DB",       "cosmetics_regulatory_db")

CONNECTION_STRING = f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}:{MYSQL_PORT}/{MYSQL_DB}"

OUTPUT_DIR = Path.cwd() / "data" / "processed"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

In [3]:
# Connetion
engine = create_engine(CONNECTION_STRING)

### Q1 — Basic extraction (all Sephora products)

In [4]:
q1 = text("""
    SELECT product_id, product_name, brand_name, product_type, price_usd, rating
    FROM sephora_products
    ORDER BY brand_name ASC, product_name ASC
    LIMIT 100
""")
df_q1 = pd.read_sql(q1, engine)
df_q1.to_csv(OUTPUT_DIR / "q1_sephora_base.csv", index=False)
print("Q1 — Basic extraction:", df_q1.shape)
df_q1.head()

Q1 — Basic extraction: (100, 6)


,product_id,product_name,brand_name,product_type,price_usd,rating
0,P472300,Capri Eau de Parfum,19-69,Fragrance,195.0,3.5714
1,P473670,Capri Eau de Parfum Travel Spray,19-69,Fragrance,30.0,3.5714
2,P473671,Fragrance Discovery Set,19-69,Fragrance,35.0,3.6364
3,P473666,Invisible Post Eau de Parfum,19-69,Fragrance,195.0,3.6250
4,P473667,Invisible Post Eau de Parfum Travel Spray,19-69,Fragrance,30.0,3.6250


### Q2 — Products with restricted ingredients

In [5]:
q2 = text("""
    SELECT product_id, product_name, brand_name, product_type,
           price_usd, rating, restricted_ingredient_count, cmr_count
    FROM sephora_products
    WHERE has_restricted_ingredient = 1
    ORDER BY restricted_ingredient_count DESC
    LIMIT 50
""")
df_q2 = pd.read_sql(q2, engine)
df_q2.to_csv(OUTPUT_DIR / "q2_restricted_products.csv", index=False)
print("Q2 — Restricted products:", df_q2.shape)
df_q2.head()

Q2 — Restricted products: (50, 8)


,product_id,product_name,brand_name,product_type,price_usd,rating,restricted_ingredient_count,cmr_count
0,P407907,Sweet Peach Eyeshadow Palette,Too Faced,Makeup,49.0,4.4938,94,0
1,P480182,REPLICA' Memory Box Perfume Set,Maison Margiela,Fragrance,39.0,3.9660,92,0
2,P482527,Scent DIARY Fragrance Discovery Set,Ellis Brooklyn,Fragrance,32.0,4.3385,90,0
3,P501965,Archive Kit Sampler Set,Commodity,Fragrance,36.0,3.4000,78,0
4,P473709,Perfume Discovery Set,Floral Street,Fragrance,24.0,3.5405,67,0


### Q3 — Multi-criteria filtering (CMR + premium + rating)

In [6]:
q3 = text("""
    SELECT product_id, product_name, brand_name, product_type,
           price_usd, rating, restricted_ingredient_count, cmr_count
    FROM sephora_products
    WHERE has_cmr = 1
      AND price_usd > 50
      AND rating IS NOT NULL
    ORDER BY cmr_count DESC, price_usd DESC
""")
df_q3 = pd.read_sql(q3, engine)
df_q3.to_csv(OUTPUT_DIR / "q3_high_risk_products.csv", index=False)
print("Q3 — High risk (CMR + premium):", df_q3.shape)
df_q3.head()

Q3 — High risk (CMR + premium): (100, 8)


,product_id,product_name,brand_name,product_type,price_usd,rating,restricted_ingredient_count,cmr_count
0,P500155,ExfoliKate All Over Glow Kit,Kate Somerville,Skincare,115.0,5.0000,18,2
1,P481339,Alpha Beta Daily Essentials - Extra Strength,Dr. Dennis Gross Skincare,Skincare,92.0,4.4615,10,2
2,P481975,Alpha Beta Daily Essentials - Universal,Dr. Dennis Gross Skincare,Skincare,92.0,4.6296,10,2
3,P481347,Clear Skin Kit Acne Treatment Kit,Paula's Choice,Skincare,75.0,3.4375,5,2
4,P481350,Complexion Clarity Kit,Paula's Choice,Skincare,64.0,4.4444,5,2


### Q4 — Aggregation by product type

In [9]:
q4 = text("""
    SELECT
        product_type,
        COUNT(*)                                                   AS total_products,
        ROUND(AVG(restricted_ingredient_count), 2)                 AS avg_restricted_ingredients,
        ROUND(AVG(cmr_count), 2)                                   AS avg_cmr,
        SUM(has_restricted_ingredient)                             AS nb_restricted_products,
        SUM(has_cmr)                                               AS nb_cmr_products,
        ROUND(SUM(has_restricted_ingredient) / COUNT(*) * 100, 1)  AS pct_restricted,
        ROUND(SUM(has_cmr) / COUNT(*) * 100, 1)                    AS pct_cmr,
        MAX(restricted_ingredient_count)                           AS max_restricted_ingredients
    FROM sephora_products
    GROUP BY product_type
    HAVING COUNT(*) > 10
    ORDER BY pct_restricted DESC
""")
df_q4 = pd.read_sql(q4, engine)
df_q4.to_csv(OUTPUT_DIR / "q4_aggregation_by_type.csv", index=False)
print("Q4 — Aggregation by type :", df_q4.shape)
df_q4

Q4 — Aggregation by type : (7, 9)


,product_type,total_products,avg_restricted_ingredients,avg_cmr,nb_restricted_products,nb_cmr_products,pct_restricted,pct_cmr,max_restricted_ingredients
0,Fragrance,1115,9.28,0.00,1054.0,0.0,94.5,0.0,92
1,Hair,1270,3.80,0.10,1064.0,119.0,83.8,9.4,25
2,Men,59,2.85,0.14,48.0,8.0,81.4,13.6,19
3,Bath & Body,371,3.44,0.03,282.0,12.0,76.0,3.2,12
4,Mini Size,266,2.89,0.02,200.0,5.0,75.2,1.9,43
5,Skincare,2286,2.52,0.11,1649.0,233.0,72.1,10.2,40
6,Makeup,2013,1.87,0.01,1302.0,16.0,64.7,0.8,94


### Q5 — Top 10 brands by restricted products

In [10]:
q5 = text("""
    SELECT
        brand_name,
        COUNT(*)                          AS total_products,
        SUM(has_restricted_ingredient)    AS nb_restricted_products,
        SUM(has_cmr)                      AS nb_cmr_products,
        ROUND(AVG(price_usd), 2)          AS avg_price
    FROM sephora_products
    GROUP BY brand_name
    HAVING SUM(has_restricted_ingredient) > 0
    ORDER BY nb_restricted_products DESC
    LIMIT 10
""")
df_q5 = pd.read_sql(q5, engine)
df_q5.to_csv(OUTPUT_DIR / "q5_top10_brands.csv", index=False)
print("Q5 — Top 10 brands:", df_q5.shape)
df_q5

Q5 — Top 10 brands: (10, 5)


,brand_name,total_products,nb_restricted_products,nb_cmr_products,avg_price
0,CLINIQUE,177,128.0,26.0,35.46
1,SEPHORA COLLECTION,201,108.0,15.0,13.29
2,Bumble and bumble,108,105.0,2.0,29.78
3,Kérastase,108,105.0,37.0,47.35
4,Dior,125,96.0,0.0,75.65
5,TOM FORD,96,91.0,1.0,159.43
6,Oribe,86,86.0,8.0,45.35
7,Charlotte Tilbury,93,81.0,1.0,42.71
8,tarte,119,67.0,2.0,25.11
9,fresh,65,62.0,0.0,42.14


### Q6 — Join Sephora vs Skincare

In [22]:
q6_brand = text("""
    SELECT
        s.brand_name,
        s.total_sephora,
        s.pct_restricted_sephora,
        s.price_avg_sephora,
        sk.total_skincare,
        sk.pct_restricted_skincare,
        sk.price_avg_skincare
    FROM (
        SELECT
            brand_name,
            COUNT(*)                                                   AS total_sephora,
            ROUND(SUM(has_restricted_ingredient) / COUNT(*) * 100, 1)  AS pct_restricted_sephora,
            ROUND(AVG(price_usd), 2)                                   AS price_avg_sephora
        FROM sephora_products
        GROUP BY brand_name
    ) s
    INNER JOIN (
        SELECT
            brand,
            COUNT(*)                                                   AS total_skincare,
            ROUND(SUM(has_restricted_ingredient) / COUNT(*) * 100, 1)  AS pct_restricted_skincare,
            ROUND(AVG(price), 2)                                       AS price_avg_skincare
        FROM skincare_products
        GROUP BY brand
    ) sk
    ON s.brand_name = sk.brand
    ORDER BY s.pct_restricted_sephora DESC
""")

df_q6_brand = pd.read_sql(q6_brand, engine)
df_q6_brand.to_csv(OUTPUT_DIR / "q6_join_brand.csv", index=False)
print("Q6 — Join Sephora vs Skincare by brand:", df_q6_brand.shape)
df_q6_brand

Q6 — Join Sephora vs Skincare by brand: (72, 7)


,brand_name,total_sephora,pct_restricted_sephora,price_avg_sephora,total_skincare,pct_restricted_skincare,price_avg_skincare
0,belif,14,100.0,35.57,18,61.1,39.44
1,La Mer,38,100.0,200.53,29,100.0,184.66
2,Erno Laszlo,3,100.0,66.67,6,66.7,69.00
3,philosophy,1,100.0,24.00,24,91.7,46.54
4,OLEHENRIKSEN,38,100.0,43.16,25,96.0,39.36
...,...,...,...,...,...,...,...
67,Moon Juice,15,26.7,45.87,2,100.0,48.50
68,HUM Nutrition,27,0.0,29.30,4,0.0,30.00
69,FOREO,8,0.0,86.62,6,0.0,15.67
70,Skin Laundry,3,0.0,15.67,5,40.0,44.20


### Q7 — Products above average risk

In [12]:
q7 = text("""
    SELECT product_id, product_name, brand_name, product_type,
           restricted_ingredient_count, cmr_count
    FROM sephora_products
    WHERE restricted_ingredient_count > (
        SELECT AVG(restricted_ingredient_count)
        FROM sephora_products
    )
    ORDER BY restricted_ingredient_count DESC
    LIMIT 30
""")
df_q7 = pd.read_sql(q7, engine)
df_q7.to_csv(OUTPUT_DIR / "q7_above_average.csv", index=False)
print("Q7 — Above average:", df_q7.shape)
df_q7.head()

Q7 — Above average: (30, 6)


,product_id,product_name,brand_name,product_type,restricted_ingredient_count,cmr_count
0,P407907,Sweet Peach Eyeshadow Palette,Too Faced,Makeup,94,0
1,P480182,REPLICA' Memory Box Perfume Set,Maison Margiela,Fragrance,92,0
2,P482527,Scent DIARY Fragrance Discovery Set,Ellis Brooklyn,Fragrance,90,0
3,P501965,Archive Kit Sampler Set,Commodity,Fragrance,78,0
4,P473709,Perfume Discovery Set,Floral Street,Fragrance,67,0


### Q8 — Skincare with CMR ingredients

In [13]:
q8 = text("""
    SELECT brand, product_name, product_type, price, rating,
           restricted_ingredient_count, cmr_count
    FROM skincare_products
    WHERE has_cmr = 1
    ORDER BY price DESC
""")
df_q8 = pd.read_sql(q8, engine)
df_q8.to_csv(OUTPUT_DIR / "q8_skincare_cmr.csv", index=False)
print("Q8 — Skincare CMR :", df_q8.shape)
df_q8

Q8 — Skincare CMR : (90, 7)


,brand,product_name,product_type,price,rating,restricted_ingredient_count,cmr_count
0,GLAMGLOW,The Ultimate Glow Set,Face Mask,169,0.0,9,1
1,OMOROVICZA,Silver Skin Savior Salicylic/Glycolic Acid Tre...,Treatment,125,5.0,5,1
2,OMOROVICZA,Midnight Radiance Mask,Face Mask,125,5.0,8,1
3,PERRICONE MD,High Potency Classics: Firming Evening Repair,Treatment,99,4.4,6,1
4,ESTÉE LAUDER,Enlighten Dark Spot Correcting Night Serum,Treatment,98,4.0,2,1
...,...,...,...,...,...,...,...
85,CLINIQUE,Fresh Pressed Daily Booster with Pure Vitamin ...,Treatment,19,4.0,1,1
86,CLINIQUE,Clarifying Lotion 4,Cleanser,16,4.4,2,1
87,CLINIQUE,Clarifying Lotion 3,Cleanser,16,4.4,2,1
88,CLINIQUE,Clarifying Lotion 1,Cleanser,16,4.5,2,1


In [15]:
# Cleaning
engine.dispose()
print("\n✓ All 8 queries have been executed and saved in data/processed/")


✓ All 8 queries have been executed and saved in data/processed/
